In [110]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)

start_date = datetime.date(2015, 1, 1)
end_date = datetime.date(2015, 1, 30)
for single_date in daterange(start_date, end_date):
    print(single_date+datetime.timedelta(1))

2015-01-02
2015-01-03
2015-01-04
2015-01-05
2015-01-06
2015-01-07
2015-01-08
2015-01-09
2015-01-10
2015-01-11
2015-01-12
2015-01-13
2015-01-14
2015-01-15
2015-01-16
2015-01-17
2015-01-18
2015-01-19
2015-01-20
2015-01-21
2015-01-22
2015-01-23
2015-01-24
2015-01-25
2015-01-26
2015-01-27
2015-01-28
2015-01-29
2015-01-30


In [176]:
def get_urls(startDate,endDate,count=200,sources=["CNN","NYT","BBC","Reuters","Fox News","Telegraph","Guardian","The Times"]):
    
    def daterange(start_date, end_date):
        for n in range(int ((end_date - start_date).days)):
            yield start_date + datetime.timedelta(n)

    er = EventRegistry()
    q = QueryArticles()
    # set news sources
    for x in sources:
        sourceUri = er.getNewsSourceUri(x)
        q.addNewsSource(sourceUri)

    # set result settings
    q.addRequestedResult(RequestArticlesInfo(page = 1, count = count,
        returnInfo = ReturnInfo(articleInfo = ArticleInfoFlags(concepts = False, categories = False, image = False))))
        
    urls = []
        
    for single_date in daterange(startDate, endDate):
        print(single_date,single_date+datetime.timedelta(1))
        # set the date limit of interest
        q.setDateLimit(single_date, single_date+datetime.timedelta(1))
        
        res = er.execQuery(q)
        
        try:
            for x in range(0,count):
                urls.append(res['articles']['results'][x]['url'])
        except:
            print(res['error'])
        
    unique_urls = list(set(urls))
    
    return unique_urls

In [177]:
start_date = datetime.date(2017, 1, 1)
end_date = datetime.date(2017, 1, 31)
urls = get_urls(start_date,end_date,200)

found username and password in settings file which will be used for making requests. Trying to login...
Event Registry host: http://eventregistry.org
2017-01-01 2017-01-02
No results match the given time constraints
2017-01-02 2017-01-03
No results match the given time constraints
2017-01-03 2017-01-04
No results match the given time constraints
2017-01-04 2017-01-05
2017-01-05 2017-01-06
2017-01-06 2017-01-07
2017-01-07 2017-01-08
2017-01-08 2017-01-09
2017-01-09 2017-01-10
2017-01-10 2017-01-11
2017-01-11 2017-01-12
2017-01-12 2017-01-13
2017-01-13 2017-01-14
2017-01-14 2017-01-15
2017-01-15 2017-01-16
2017-01-16 2017-01-17
2017-01-17 2017-01-18
2017-01-18 2017-01-19
2017-01-19 2017-01-20
2017-01-20 2017-01-21
2017-01-21 2017-01-22
2017-01-22 2017-01-23
2017-01-23 2017-01-24
2017-01-24 2017-01-25
2017-01-25 2017-01-26
2017-01-26 2017-01-27
2017-01-27 2017-01-28
2017-01-28 2017-01-29
2017-01-29 2017-01-30
2017-01-30 2017-01-31


In [178]:
len(urls)

4992

In [179]:
urls[:10]

['http://www.reuters.com/video/2014/03/21/mixed-reaction-to-crimeas-annexation?videoId=297219996&videoChannel=2602',
 'http://www.foxnews.com/us/2017/01/07/airport-shooting-raises-questions-about-guns-in-baggage.html',
 'http://www.foxnews.com/us/2017/01/26/man-admits-to-luring-gay-man-into-woods-fatally-beating-him.html',
 'http://www.reuters.com/article/us-jpmorgan-dimon-compensation-idUSKBN15336N',
 'http://www.cnn.com/2017/01/05/weather/winter-storm-south-snow/index.html',
 'http://www.telegraph.co.uk/business/2017/01/15/mauritius-payment-knocks-fundsmith-profits-despite-surge-new/',
 'http://www.bbc.co.uk/news/uk-northern-ireland-38726562',
 'http://www.reuters.com/article/us-usa-congress-price-hearing-idUSKBN1582MP',
 'http://www.telegraph.co.uk/business/2017/01/28/collusion-suspected-bt-scandal/',
 'http://www.cnn.com/2017/01/29/politics/trump-travel-ban-congress-reaction/index.html']

In [180]:
import pickle

with open('er_urls_012017.pkl', 'wb') as handle:
    pickle.dump(urls, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def SVM_evaluate(self):
    import pandas as pd
    import numpy as np
    from sklearn import svm
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import confusion_matrix

    json_dat = [doc for doc in self._db.clean_data.find()]
    df_dat = pd.DataFrame(json_dat)
    Y = np.array(df_dat['True'])

    df_dat.drop(['_id','body_length','resource','title_compounds','url','True'], axis=1, inplace=True)

    X = np.array(df_dat)


    K = 10
    skf = StratifiedKFold(n_splits=K, random_state=None, shuffle=True)

    acc = []
    for train_index, test_index in skf.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        # Train classifier
        clf = svm.SVC(probability=True)
        clf.fit(X_train, Y_train)  

        Y_pred = clf.predict(X_test)

        Cfold = confusion_matrix(Y_test,Y_pred)
        self.classifier_performance['confusion'] += Cfold
        acc.append(sum(np.diag(Cfold))/sum(sum(Cfold)))

    acc = np.array(acc)
    self.classifier_performance['accuracy_mean'] = np.mean(acc)
    self.classifier_performance['accuracy_std'] = np.std(acc)
    
def SVM_fit(self):
    import pandas as pd
    import numpy as np
    from sklearn import svm
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import confusion_matrix

    json_dat = [doc for doc in self._db.clean_data.find()]
    df_dat = pd.DataFrame(json_dat)
    Y = np.array(df_dat['True'])

    df_dat.drop(['_id','body_length','resource','title_compounds','url','True'], axis=1, inplace=True)

    X = np.array(df_dat)

    # Train classifier
    self.clf = svm.SVC(probability=True)
    self.clf.fit(X, Y)  